In [22]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)


In [60]:
sample_size = 100000

query = '''
    SELECT RAND() as r, id, creation_date FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
    ORDER BY r
    LIMIT ''' + str(sample_size)


df = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df.to_csv('quarter_posts.csv')

In [61]:
def creationDateToDay(date):
    return pd.to_datetime(date)

df['creation_date'] = df['creation_date'].map(creationDateToDay)
sr = pd.Series(df['creation_date'])
result = sr.dt.quarter
df = result.to_frame()
df

,creation_date
0,2
1,1
2,2
3,1
4,3
...,...
99995,2
99996,2
99997,2
99998,3


In [62]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=df.creation_date,
    histnorm='percent',
    name='Posts',
    marker_color='#330C73',
    opacity=0.75
))

fig.update_layout(
    title_text='Number of posts based on year quarter', # title of plot
    xaxis_title_text='Quarter of year', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.5, # gap between bars of adjacent location coordinates
)

fig.show()